<a href="https://colab.research.google.com/github/sankeawthong/Project-1-Lita-Chatbot/blob/main/%5B20250524%5D%20Trust-FedAvg%20MLP-LSTM%20with%2030%20Rounds%20%26%205%20clients%20on%20WSN-DS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Trust-FedAvg MLP-LSTM with 30 Communications Rounds & 5 clients on WSN-DS**

In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix, roc_auc_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, InputLayer
from keras.utils import to_categorical
from keras.regularizers import l2
from imblearn.over_sampling import SMOTE
from scipy.spatial.distance import cosine

In [2]:
# ----------------------------
# Preprocessing
# ----------------------------
df = pd.read_csv("dataset_WSN-DS.csv").dropna()
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = LabelEncoder().fit_transform(df[col])

X, y = df.drop('Class', axis=1), df['Class']
X = StandardScaler().fit_transform(X)
X, y = SMOTE(random_state=42).fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101, stratify=y)

In [3]:
# ----------------------------
# Federated Setup
# ----------------------------
num_clients = 5
rounds = 30
alpha = (0.4, 0.4, 0.2)

client_X = np.array_split(X_train, num_clients)
client_y = np.array_split(y_train, num_clients)

X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
y_test_cat = to_categorical(y_test)
num_classes = y_test_cat.shape[1]
input_shape = (X_test.shape[1], 1)

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


In [4]:
# ----------------------------
# Model Definition
# ----------------------------
def build_model(input_shape, num_classes):
    model = Sequential([
        InputLayer(input_shape=input_shape),
        LSTM(64, activation='tanh', kernel_regularizer=l2(0.01)),
        Dropout(0.3),
        Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
        Dropout(0.3),
        Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def cosine_sim(w1, w2):
    f1 = np.concatenate([layer.flatten() for layer in w1])
    f2 = np.concatenate([layer.flatten() for layer in w2])
    return 1 - cosine(f1, f2)

def update_stability(w_now, w_prev):
    if w_prev is None:
        return 1.0
    return cosine_sim(w_now, w_prev)

def normalize(val, min_val, max_val):
    return (val - min_val) / (max_val - min_val + 1e-8)

# ----------------------------
# Trust-FedAvg Training Loop
# ----------------------------
log = []
trust_history = {i: [] for i in range(num_clients)}
prev_weights = {i: None for i in range(num_clients)}

global_model = build_model(input_shape, num_classes)
global_weights = global_model.get_weights()

for r in range(rounds):
    round_start = time.time()
    updates, val_losses, trust_scores = {}, {}, {}

    for i in range(num_clients):
        model = build_model(input_shape, num_classes)
        model.set_weights(global_weights)

        X_c = client_X[i].reshape(client_X[i].shape[0], client_X[i].shape[1], 1)
        y_c = to_categorical(client_y[i], num_classes=num_classes)
        model.fit(X_c, y_c, epochs=1, batch_size=32, verbose=0)

        updates[i] = model.get_weights()
        loss, _ = model.evaluate(X_test, y_test_cat, verbose=0)
        val_losses[i] = loss

    # Normalize validation losses
    min_l, max_l = min(val_losses.values()), max(val_losses.values())
    for i in range(num_clients):
        S_i = cosine_sim(updates[i], global_weights)
        L_i = normalize(val_losses[i], min_l, max_l)
        U_i = update_stability(updates[i], prev_weights[i])
        trust_scores[i] = alpha[0]*S_i + alpha[1]*(1 - L_i) + alpha[2]*U_i
        trust_history[i].append(trust_scores[i])
        prev_weights[i] = updates[i]

    total_trust = sum(trust_scores.values())
    new_weights = [sum(trust_scores[i] * np.array(updates[i][j]) for i in updates) / total_trust for j in range(len(global_weights))]
    global_model.set_weights(new_weights)

    y_pred = global_model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y_test_cat, axis=1)
    acc = accuracy_score(y_true, y_pred_classes)
    prec = precision_score(y_true, y_pred_classes, average='weighted')
    rec = recall_score(y_true, y_pred_classes, average='weighted')
    f1 = f1_score(y_true, y_pred_classes, average='weighted')
    auc = roc_auc_score(y_test_cat, y_pred, multi_class='ovr')
    round_time = time.time() - round_start

    log.append({
        'Round': r+1,
        'Accuracy': acc,
        'Precision': prec,
        'Recall': rec,
        'F1-Score': f1,
        'AUC': auc,
        'RoundTime(s)': round_time,
        'MinTrust': min(trust_scores.values()),
        'MaxTrust': max(trust_scores.values()),
        'AvgTrust': np.mean(list(trust_scores.values()))
    })

    print(f"Round {r+1}: Acc={acc:.4f}, F1={f1:.4f}, AUC={auc:.4f}, Time={round_time:.2f}s")
    print("Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred_classes))
    print(classification_report(y_true, y_pred_classes))

# ----------------------------
# Save Logs
# ----------------------------
df_log = pd.DataFrame(log)
df_log.to_csv("trustfed_training_log.csv", index=False)
print("Training log saved to trustfed_training_log.csv")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


10628/10628 ━━━━━━━━━━━━━━━━━━━━ 34s 3ms/step
Round 1: Acc=0.6421, F1=0.5890, AUC=0.9227, Time=585.70s
Confusion Matrix:
[[64986   222     4  2730    71]
 [ 7375 50705   147  9786     0]
 [ 6341 42508  1488 17676     0]
 [ 8519  3757  6393 49345     0]
 [15618   412     0   149 51834]]
              precision    recall  f1-score   support

           0       0.63      0.96      0.76     68013
           1       0.52      0.75      0.61     68013
           2       0.19      0.02      0.04     68013
           3       0.62      0.73      0.67     68014
           4       1.00      0.76      0.86     68013

    accuracy                           0.64    340066
   macro avg       0.59      0.64      0.59    340066
weighted avg       0.59      0.64      0.59    340066



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


10628/10628 ━━━━━━━━━━━━━━━━━━━━ 34s 3ms/step
Round 2: Acc=0.6511, F1=0.5967, AUC=0.9164, Time=598.18s
Confusion Matrix:
[[66339   170     2  1348   154]
 [21088 38258   162  7860   645]
 [ 6895 43613  1541 15964     0]
 [ 9062   641  9278 49033     0]
 [ 1749     0     0     4 66260]]
              precision    recall  f1-score   support

           0       0.63      0.98      0.77     68013
           1       0.46      0.56      0.51     68013
           2       0.14      0.02      0.04     68013
           3       0.66      0.72      0.69     68014
           4       0.99      0.97      0.98     68013

    accuracy                           0.65    340066
   macro avg       0.58      0.65      0.60    340066
weighted avg       0.58      0.65      0.60    340066



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


...

10628/10628 ━━━━━━━━━━━━━━━━━━━━ 33s 3ms/step
Round 28: Acc=0.6913, F1=0.6875, AUC=0.9295, Time=616.46s
Confusion Matrix:
[[59192   349    89  8343    40]
 [ 3835 30906 22078 11194     0]
 [ 1158  9078 48287  9490     0]
 [ 5634    51  8177 54152     0]
 [17485  6474     0  1498 42556]]
              precision    recall  f1-score   support

           0       0.68      0.87      0.76     68013
           1       0.66      0.45      0.54     68013
           2       0.61      0.71      0.66     68013
           3       0.64      0.80      0.71     68014
           4       1.00      0.63      0.77     68013

    accuracy                           0.69    340066
   macro avg       0.72      0.69      0.69    340066
weighted avg       0.72      0.69      0.69    340066



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


10628/10628 ━━━━━━━━━━━━━━━━━━━━ 32s 3ms/step
Round 29: Acc=0.4300, F1=0.3653, AUC=0.8890, Time=580.10s
Confusion Matrix:
[[67695   145    41   130     2]
 [42560 24557   169   727     0]
 [15495 37754  6162  8602     0]
 [11178  6082  5668 45086     0]
 [64903   317     0    78  2715]]
              precision    recall  f1-score   support

           0       0.34      1.00      0.50     68013
           1       0.36      0.36      0.36     68013
           2       0.51      0.09      0.15     68013
           3       0.83      0.66      0.74     68014
           4       1.00      0.04      0.08     68013

    accuracy                           0.43    340066
   macro avg       0.61      0.43      0.37    340066
weighted avg       0.61      0.43      0.37    340066



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


10628/10628 ━━━━━━━━━━━━━━━━━━━━ 35s 3ms/step
Round 30: Acc=0.5856, F1=0.5411, AUC=0.8877, Time=619.20s
Confusion Matrix:
[[66847   108     1   992    65]
 [28048 31880   165  7920     0]
 [10330 41768  1513 14402     0]
 [ 9201  1038 12358 45417     0]
 [13264   536     0   734 53479]]
              precision    recall  f1-score   support

           0       0.52      0.98      0.68     68013
           1       0.42      0.47      0.44     68013
           2       0.11      0.02      0.04     68013
           3       0.65      0.67      0.66     68014
           4       1.00      0.79      0.88     68013

    accuracy                           0.59    340066
   macro avg       0.54      0.59      0.54    340066
weighted avg       0.54      0.59      0.54    340066

Training log saved to trustfed_training_log.csv
